In [25]:
import covid19sim
import pandas as pd
import dill
from collections import defaultdict
import datetime
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
filename = "../src/logs2/tracker_data_n_100_seed_0_20200504-225934.pkl"

In [5]:
data = dill.load(open(filename,"rb"))

In [6]:
humans = {}
for i in data['human_monitor'].keys():
    row = defaultdict(list)
    for c,j in enumerate(data['human_monitor'][i]):
        for k in j.keys():
            row[k].append(j[k])

    # index = (timestamp, attr)
    for k in row.keys():
        humans[(i,k)] = row[k]
    
humans = pd.DataFrame.from_dict(humans, orient="index")
humans.index = pd.MultiIndex.from_tuples(humans.index)

### Select chain of infections to track
Use `data['infection_monitor']` to select the chain of infections. If A infected B, B infected C. Note their numbers.

In [7]:
data['infection_monitor']

[['human:8', 'human:6', datetime.date(2020, 3, 1)],
 ['human:84', 'human:99', datetime.date(2020, 3, 1)],
 ['human:8', 'human:7', datetime.date(2020, 3, 1)],
 ['human:36', 'human:37', datetime.date(2020, 3, 2)],
 ['human:84', 'human:102', datetime.date(2020, 3, 2)],
 ['human:24', 'human:21', datetime.date(2020, 3, 2)],
 ['human:8', 'human:9', datetime.date(2020, 3, 3)],
 ['human:99', 'human:72', datetime.date(2020, 3, 4)],
 ['human:36', 'human:16', datetime.date(2020, 3, 4)],
 ['human:7', 'human:1', datetime.date(2020, 3, 4)],
 ['human:37', 'human:44', datetime.date(2020, 3, 5)],
 ['human:99', 'human:85', datetime.date(2020, 3, 5)],
 ['human:21', 'human:18', datetime.date(2020, 3, 5)],
 ['human:6', 'human:51', datetime.date(2020, 3, 5)],
 ['human:102', 'human:95', datetime.date(2020, 3, 5)],
 ['human:6', 'human:23', datetime.date(2020, 3, 6)],
 ['human:24', 'human:25', datetime.date(2020, 3, 6)],
 ['human:9', 'human:10', datetime.date(2020, 3, 7)],
 ['human:16', 'human:13', datetime.da

In [18]:
def _color_cell(val):
    color = "white"
    if val == "positive":
        color = "red"
#     if val == 0.0:
#         color = "green"
    return f"background-color: {color}"

def spy(id1, id2, id3):
    pd.set_option('display.max_rows', None)
    df = humans.loc[(slice(None), ("infection_timestamp","risk", "test_result", "n_symptoms", "risk_level", "rec_level")), [id1-1,id2-1,id3-1]]
    df = df.style.applymap(_color_cell)
    return df


In [19]:
# track 8,6,51 (check from data['infection_monitor'])
# spy(8,6,51)
spy(36, 37, 44)

# TODOs
# color test_results with 'red' and 'yellow' of lower transparency
# color risk_levels with one of 16 colors from a color map
# color recommendation_level with one of 4 colors - "red", "orange", "yellow", "green"
# have an option to select what to view i.e. test_results, symptoms, risk, risk_level, etc. 

In [30]:
# plot risk trajectory of people until they are deemed positive
# once someone is positive, the risk only goes to 1.0 and then 0.0 when it recovers
df = humans.loc[(slice(None), ("risk")),[35, 36, 43]]
df[, :]
# .plot(figsize=(15,10))

# TODOs (plot risk for chains)
# plot the time (as a dot) when someone is infected in the chain 
# plot the time when someone gets a positive test result - this marks the end of the plot  (fill the 0s with NaN)
# have days since outbreak on the x-axis
# have chains of 4-5 people

,,35,36,43
2020-02-29,risk,0.01,0.01,0.01
2020-03-01,risk,0.01,0.01,0.01
2020-03-02,risk,0.01,0.01,0.01
2020-03-03,risk,0.01,0.01,0.01
2020-03-04,risk,0.01,0.01,0.01
2020-03-05,risk,0.01,0.01,0.01
2020-03-06,risk,0.01,0.01,0.01
2020-03-07,risk,0.01,0.01,0.01
2020-03-08,risk,0.01,0.01,0.01
2020-03-09,risk,0.01,0.01,0.01
